# Mesh to Mesh Registration Example

ITK natively supports image-to-image registration, which is a common operation for medical images with symmetry. Another common method of storing 3D volumetric data is to represent volume surfaces with meshes. In this example we seek to register two meshes using various ITK metrics and optimization techniques.

Registration classes are defined in the Python `hasi` submodule and built on top of the ITK Python wrapping. The `MeanSquaresRegistrar` and `DiffeoRegistrar` classes apply registration techniques to images derived from mesh inputs, while the `PointSetEntropyRegistrar` aims to register meshes via point set entropy metrics. Mesh registration is carried out with each class in this notebook on sample bone femur mesh data downloaded to the `examples/Data` folder.

This notebook requires the following modules, which can be either acquired via `pip` or built alongside the ITK `master` branch:
- [ITK](https://github.com/InsightSoftwareConsortium/ITK/)
- [ITKBoneEnhancement](https://github.com/InsightSoftwareConsortium/ITKBoneEnhancement)
- [ITKMeshToPolyData](https://github.com/InsightSoftwareConsortium/ITKMeshToPolyData)
- [ITKWidgets](https://github.com/InsightSoftwareConsortium/itkwidgets)

In [1]:
# Update sys.path to reference src/ modules
import os
import sys
import copy
import importlib
from urllib.request import urlretrieve

import itk
from itkwidgets import view, checkerboard, compare
from ipywidgets import FloatProgress, Label, HBox, VBox, FloatText, ColorPicker, Button
PATTERN_COUNT = 5

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
os.makedirs('Input', exist_ok=True)
os.makedirs('Output', exist_ok=True)

In [3]:
MESH_TO_USE = '901-R'
TARGET_MESH_FILE = f'Input/meshes/{MESH_TO_USE}-mesh.vtk'#f'Input/{MESH_TO_USE}-mesh.vtk'
TEMPLATE_MESH_FILE = f'Input/906-R-atlas-clean.obj'

MEANSQUARES_OUTPUT_FILE = f'Output/{MESH_TO_USE}-meansquares-registered.obj'
DIFFEO_OUTPUT_FILE = f'Output/{MESH_TO_USE}-diffeo-registered.obj'
POINTSET_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-registered.obj'
POINTSET_RESAMPLED_OUTPUT_FILE = f'Output/{MESH_TO_USE}-pointset-resampled.obj'

In [4]:
# Download meshes
if not os.path.exists(TARGET_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5f9daaba50a41e3d1924dae9/download'
    urlretrieve(url, TARGET_MESH_FILE)
if not os.path.exists(TEMPLATE_MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/608b006d2fa25629b970f139/download'
    urlretrieve(url, TEMPLATE_MESH_FILE)

In [4]:
template_mesh = itk.meshread(TEMPLATE_MESH_FILE, itk.F)
target_mesh = itk.meshread(TARGET_MESH_FILE, itk.F)

## Compare images with ITKWidgets

We can use `view`, `compare`, and `checkerboard` to inspect mesh and image data.

In [6]:
#view(geometries=[template_mesh,target_mesh])

## Mesh-To-Image Conversion in the Base Class
Python registration classes inheriting from `MeshToMeshRegistrar` implement unique registration algorithms. The base class includes common definitions for 3D mesh-to-mesh registration, abstract methods, and a mesh-to-image conversion method.

The `mesh_to_image` function takes in a 3D mesh and converts it into an ITK 3D image object. The spacing, origin, and size of the 3D image may be calculated from the minimum bounding box of the mesh or set from a reference image.

If a list of meshes is passed to `mesh_to_image` then the output images will each be set with the same size, spacing, and origin so that every mesh is fully contained within the image region.

In [7]:
from src.hasi.hasi.meshtomeshregistrar import MeshToMeshRegistrar
registrar = MeshToMeshRegistrar()

In [8]:
target_image, template_image = registrar.mesh_to_image([target_mesh, template_mesh])

Comparing the meshes generated from the given meshes, we see that the two bone images are generally similar but do not exactly line up together. Registration will translate one mesh so that the two bones better coincide.

In [9]:
#compare(template_image,target_image)

In [10]:
#checkerboard(template_image, target_image, pattern=PATTERN_COUNT)

## Point Set Downsizing in the Base Class

The base `MeshToMeshRegistrar` class also provides decimation functionality for uniform random point set sampling from a given mesh, primarily used to improve performance for point set based registration of dense meshes. Note that uniform random sampling may not be suitable for all shapes, in which case application-specific resampling should be carried out externally prior to registration.

For subsequent registration in this notebook we will rely on a template atlas with approximately 4,000 points which does not require decimation.

In [11]:
target_points_reduced = \
    registrar.randomly_sample_mesh_points(mesh=target_mesh, sampling_rate=0.01)

In [12]:
#view(geometries=[target_mesh],point_sets=[target_points_reduced])

## Run Mean Squares Image Registration

The `MeanSquaresRegistrar` class converts meshes to images and runs [Broyden-Fletcher-Goldfarb-Shanno Optimization](https://itk.org/Doxygen/html/classitk_1_1LBFGSBOptimizerv4.html) on a [BSplineTransform](https://itk.org/Doxygen/html/classitk_1_1BSplineTransform.html) to iteratively reduce the mean square error. The resulting transform is then applied to resample the target mesh into the template mesh domain.

Progress is shown with an itkwidgets display via hooks into the ITK event-observer system. The resultant mesh is returned as an object in the Python environment and may be optionally written out to a file. Iteration updates may also be printed to the output window with the optional `verbose` flag.

In [13]:
from src.hasi.hasi.meansquaresregistrar import MeanSquaresRegistrar

In [14]:
# Must instantiate a registration object to initialize optimizers
registrar = MeanSquaresRegistrar()

In [15]:
progress = FloatProgress(
        min=0.0,
        max=150.0,
        step=1
    )
box = HBox([
    Label('Register images'),
    progress
])
box

In [16]:
def update_progress():
    progress.value = registrar.optimizer.GetCurrentIteration()
registrar.optimizer.AddObserver(itk.IterationEvent(), update_progress)

0

In [17]:
(transform_result, mesh_result) = registrar.register(template_mesh,
                                             target_mesh,
                                             filepath=MEANSQUARES_OUTPUT_FILE,
                                             verbose=True,
                                             num_iterations=200)

0 0.4953344442828443 0.0
0 0.3679928041625301 0.0
0 0.3679928041625301 0.0
1 0.1382597111142386 0.002047170989457994
1 0.1382597111142386 0.002047170989457994
2 0.09620259423214085 0.001261876484024073
2 0.09620259423214085 0.001261876484024073
3 0.0810644365222312 0.0011402423999786182
3 0.0810644365222312 0.0011402423999786182
4 0.05506265821062781 0.0008026746053761812
4 0.05506265821062781 0.0008026746053761812
5 0.036229100368238225 0.0005999657307756083
5 0.036229100368238225 0.0005999657307756083
6 0.02201122948495042 0.0006569240263049952
6 0.02201122948495042 0.0006569240263049952
7 0.019790912974060545 0.000227595005185355
7 0.019790912974060545 0.000227595005185355
8 0.017138213201441897 0.00012848767601378967
8 0.017138213201441897 0.00012848767601378967
9 0.01643926158089219 0.00011380109594120822
9 0.01643926158089219 0.00011380109594120822
10 0.015742424102559768 0.000229305332514829
10 0.015742424102559768 0.000229305332514829
11 0.015332139820951112 9.86042590450415e-0

100 0.007166497278198204 3.225428203746481e-05
100 0.007166497278198204 3.225428203746481e-05
101 0.007149650939008197 1.2455706971964613e-05
101 0.007149650939008197 1.2455706971964613e-05
102 0.007121663744128236 1.2982402927792889e-05
102 0.007121663744128236 1.2982402927792889e-05
103 0.007107483587565755 1.609346523245832e-05
103 0.007107483587565755 1.609346523245832e-05
104 0.007092042896733735 2.3113347269329504e-05
104 0.007092042896733735 2.3113347269329504e-05
105 0.0070876421136930245 3.544249173464179e-05
105 0.0070876421136930245 3.544249173464179e-05
106 0.0070856698373387254 1.011908203672434e-05
106 0.0070856698373387254 1.011908203672434e-05
107 0.007076003884998143 9.5015908970922e-06
107 0.007076003884998143 9.5015908970922e-06
108 0.0070623753300841675 1.2760766981931491e-05
108 0.0070623753300841675 1.2760766981931491e-05
109 0.007066499924291111 1.4079264563422209e-05
109 0.007056548175037092 1.4079264563422209e-05
109 0.007056548175037092 1.4079264563422209e-05


Comparison of the resulting mesh with the target shows successful registration.

In [18]:
#view(geometries=[mesh_result,target_mesh])

## Diffeomorphic Registration

The `DiffeoRegistrar` class converts meshes to images and performs registration using the [Diffeomorphic Demons registration algorithm](https://itk.org/Doxygen/html/classitk_1_1DiffeomorphicDemonsRegistrationFilter.html). The resultant deformation field is then applied to resample the target mesh into the template mesh domain.

Custom observers may print out iteration data accessed via the `registrar.filter` object.

In [19]:
from src.hasi.hasi.diffeoregistrar import DiffeoRegistrar

In [20]:
registrar = DiffeoRegistrar()

In [21]:
diffeoProgress = FloatProgress(
        min=0.0,
        max=200.0,
        step=1
    )
diffeoBox = HBox([
    Label('Register images'),
    diffeoProgress
])
diffeoBox

In [22]:
def update_diff_progress():
    diffeoProgress.value = registrar.filter.GetElapsedIterations()

registrar.filter.AddObserver(itk.IterationEvent(),update_diff_progress)

(transform_result, mesh_result) = registrar.register(template_mesh,
                                        target_mesh,
                                        filepath=DIFFEO_OUTPUT_FILE,
                                        verbose=True)

1.7976931348623157e+308
0.4953344445027531
0.4919978146151957
0.49014250375875607
0.4883152615673534
0.4876264321046578
0.4863758875349746
0.4848329850901018
0.4832380374425963
0.4816613820019586
0.47992311701764906
0.47819615858812126
0.4764180158719083
0.4746375325575763
0.4725753735317665
0.47054456107198983
0.4683531446736118
0.46619317072217226
0.4638398448774739
0.4617263143098048
0.45963088709668165
0.45765992990626214
0.45564904932109535
0.45405493940994296
0.4526567041931353
0.45126435618474847
0.4499785970781415
0.4486827830549383
0.4475157724666353
0.44628145441325595
0.4450578744943134
0.4438195343732895
0.4425877546195735
0.441339593969622
0.4401098214527867
0.4388818595160368
0.43749092474381784
0.43564875311473567
0.43370194230716685
0.4311092442088367
0.4285241607433405
0.42582081376755554
0.4231977163312322
0.4205323448942155
0.4176938430560469
0.41429447935831476
0.4105150579147669
0.4078069432083754
0.4054533839896938
0.4031936651465812
0.4008872988176601
0.398616092

Compare the translated image to the target image.

In [23]:
#view(geometries=[mesh_result, target_mesh])

# Entropy-based Registration

The `PointSetEntropyRegistrar` class registers two 3D meshes by computing the transform which minimizes entropy measures between the two point clouds. In this example we substitute a [`EuclideanDistancePointSetToPointSetMetric`](https://itk.org/Doxygen/html/classitk_1_1EuclideanDistancePointSetToPointSetMetricv4.html) to compare the two clouds.

In [5]:
from src.hasi.hasi.pointsetentropyregistrar import PointSetEntropyRegistrar
registrar = PointSetEntropyRegistrar()

In [6]:
progress = FloatProgress(
        min=0.0,
        max=2000.0,
        step=1
    )
progressBox = HBox([
    Label('Register images'),
    progress
])
progressBox

In [7]:
def update_progress():
    progressBox.value = registrar.optimizer.GetCurrentIteration()

registrar.optimizer.AddObserver(itk.IterationEvent(),update_progress)

0

In [8]:
import glob

mesh_files = glob.glob('Input\\meshes\\*.vtk')
mesh_files.sort()

In [9]:
def set_color_point_data(target):
    target.GetPointData().Reserve(target.GetNumberOfPoints())
    data = itk.array_view_from_vector_container(target.GetPointData())
    for i in range(0,data.shape[0]):
        data[i] = i

In [10]:
def set_euclidean_distance_point_data(template,target):
    assert(template.GetNumberOfPoints() == target.GetNumberOfPoints())
    
    target.GetPointData().Reserve(target.GetNumberOfPoints())
    data = itk.array_view_from_vector_container(target.GetPointData())
    
    for idx in range(target.GetNumberOfPoints()):
        data[idx] = (sum([(target.GetPoint(idx)[i] - template.GetPoint(idx)[i]) ** 2 for i in range(3)])) ** 0.5

In [11]:
mesh_results = list()
for idx in range(len(mesh_files[0:6])):
    target_mesh = itk.meshread(mesh_files[idx], itk.F)
    metric = itk.EuclideanDistancePointSetToPointSetMetricv4[itk.PointSet[itk.F,3]].New()

    registrar = PointSetEntropyRegistrar()
    (transform_result, mesh_result) = registrar.register(
                           template_mesh=template_mesh,
                           target_mesh=target_mesh,
                           metric=metric,
                           #filepath=POINTSET_OUTPUT_FILE,
                           verbose=True,
                           learning_rate=1.0,
                           max_iterations=300,
                           resample_from_target=True)
    mesh_results.append(mesh_result)

0 0.0
0 0.19892726420670154
1 0.14712119557032127
2 0.1209723401741411
3 0.10546027404054423
4 0.09453211848605567
5 0.0865709875012609
6 0.08074609606089309
7 0.07625148649594239
8 0.07271413389767949
9 0.06994671658933833
10 0.06770569577898226
11 0.0658732536652061
12 0.06433437845178984
13 0.06298979971993525
14 0.061877716184758944
15 0.060910117859481236
16 0.060051039827443434
17 0.05925875903154527
18 0.05856013493519642
19 0.05792393711771744
20 0.05732683239187792
21 0.05676079622633956
22 0.05622828758331684
23 0.05574021636346727
24 0.05528233530859789
25 0.05486758008175804
26 0.05448110548736075
27 0.05409677020036218
28 0.053728646759727675
29 0.0533619375482652
30 0.05301457891526785
31 0.052683667273763245
32 0.05235683490480732
33 0.05204278716345334
34 0.05173471964203959
35 0.05144717874401892
36 0.05114926979778699
37 0.05087721770486593
38 0.05061948810095063
39 0.050358958409998836
40 0.05009400323188594
41 0.049854402889699445
42 0.04961280117472077
43 0.0493763

21 0.05356506466317467
22 0.05235989899616003
23 0.051186069543460784
24 0.05006526597175583
25 0.04900487240341479
26 0.047979250185478335
27 0.046981427297365375
28 0.04604650089216797
29 0.04513618488734077
30 0.044220167407219846
31 0.0433820493049769
32 0.04256877082826683
33 0.041780515003636914
34 0.041018431550501694
35 0.04029334332724751
36 0.03962007474716133
37 0.03896099606047065
38 0.038313998384639135
39 0.03768782140141312
40 0.03710618942380942
41 0.036533814072203775
42 0.0359808060038661
43 0.03545676215326713
44 0.03495906086470925
45 0.03446338279872415
46 0.03397943338829701
47 0.03352633374988195
48 0.03310265426208552
49 0.03267789868111375
50 0.03227997159095161
51 0.03189211656119626
52 0.031514518277943916
53 0.03115622330604167
54 0.03080482856018285
55 0.03045935647281418
56 0.030122588527854915
57 0.029800905759109663
58 0.02948920587800176
59 0.02918334497995853
60 0.028899527341193387
61 0.02863632633193493
62 0.028377617494743986
63 0.028127527013616874

43 0.061196835141383875
44 0.06116126823292393
45 0.061129730426110106
46 0.06110464157960968
47 0.06107683551450801
48 0.06104930702540762
49 0.06102540160626269
50 0.06100194743175975
51 0.060986540289027176
52 0.06097258932487023
53 0.06095227238327838
54 0.06093303813557949
55 0.06091194672027191
56 0.060893935776053475
57 0.06087653010442453
58 0.06085870576138507
59 0.060843600983779327
60 0.06082921757384729
61 0.06081598408835482
62 0.060797367758825875
63 0.06077954745287794
64 0.060762771293073815
65 0.060752148654185927
66 0.06074091751292499
67 0.060730180090515486
68 0.06072024595755011
69 0.06070254163322645
70 0.060671436770530066
71 0.06064687483200349
72 0.060623152011977974
73 0.06060279610578802
74 0.06058013253734293
75 0.06055777991780797
76 0.06053695481005727
77 0.06052027826688989
78 0.060500660748083836
79 0.060483167182843495
80 0.06046561274932892
81 0.06045225374250944
82 0.06043614147040818
83 0.060424955941636285
84 0.06041391043266089
85 0.060404473129043

71 0.03181342752131577
72 0.03166737574343921
73 0.031525211118175696
74 0.031377490062900894
75 0.03123465424087817
76 0.03109453304232531
77 0.030967624744994055
78 0.030843814628809915
79 0.03072325799404853
80 0.03060645238653649
81 0.030494456849267215
82 0.030391260056949757
83 0.030302635101216453
84 0.03021551083200736
85 0.030131166211157395
86 0.030041643873247155
87 0.029962705054434526
88 0.029881802470015872
89 0.029795438050144677
90 0.029713368408038614
91 0.02963723903591683
92 0.029563575405426312
93 0.02949672222742915
94 0.029433227331915945
95 0.029378552350330416
96 0.02933546241934525
97 0.02929002918778826
98 0.02924404095504026
99 0.029196329318336878
100 0.029148775332062272
101 0.029107879366494175
102 0.029066785858454867
103 0.029026678047469202
104 0.02898316542700085
105 0.028939745038064974
106 0.028900444288228043
107 0.028862807929732847
108 0.028828550450589175
109 0.028798365292245394
110 0.0287629172732394
111 0.028726899846240325
112 0.0286951584682

100 0.04983697822225542
101 0.04977662087006483
102 0.049716431644936374
103 0.049652437909344
104 0.0495852074517919
105 0.049519708064472465
106 0.04945404034235945
107 0.04939534262176475
108 0.04933506508518026
109 0.04927674711516869
110 0.04922383848402841
111 0.04917083454497969
112 0.04911678505003237
113 0.04906094521727614
114 0.04900428828770237
115 0.048951800780979826
116 0.04890098577172329
117 0.04885318512056262
118 0.0488070240451736
119 0.04876293436771536
120 0.04871920652238266
121 0.04867764056381561
122 0.048635795780521
123 0.04859419316172583
124 0.04855008284328305
125 0.048502774871014
126 0.04845933424699928
127 0.048418350377955825
128 0.04837867836679199
129 0.048342338518681416
130 0.048306657802456986
131 0.0482724677624293
132 0.04823712648063659
133 0.0482054786634618
134 0.048175046980477865
135 0.04814658145700093
136 0.04811624652116881
137 0.04808840405166347
138 0.04806177219104949
139 0.048034756442161854
140 0.04800641196425632
141 0.047977224970

114 0.023774578911635695
115 0.023712004646399307
116 0.0236405116260296
117 0.023573761781110057
118 0.02351176909553266
119 0.023450458395551606
120 0.02339064939683481
121 0.023340815296324285
122 0.02329369939244305
123 0.023246210461100026
124 0.023200569070699797
125 0.0231556408324954
126 0.023114124465464483
127 0.02307463658075084
128 0.02303600659539042
129 0.022999721324801763
130 0.022959154364025087
131 0.02291777435864794
132 0.022876181138473655
133 0.02283371739882351
134 0.02279932515948362
135 0.02276647475889201
136 0.022732321361447518
137 0.02269742005534288
138 0.022665077111999547
139 0.02263128288085727
140 0.022598797517599353
141 0.02256549279010841
142 0.022532290682824984
143 0.022497641502533222
144 0.022465578681106928
145 0.022437363579064843
146 0.02240934830163372
147 0.02238326263724591
148 0.02235974372235756
149 0.022331004759652325
150 0.022302638544564095
151 0.022276101645588547
152 0.022251781628926055
153 0.022229451622054825
154 0.0222080766837

In [12]:
# TODO procrustes alignment after resampling for accurate distances
align_filter = itk.MeshProcrustesAlignFilter[type(template_mesh),type(template_mesh)].New()

align_filter.SetUseInitialAverageOff()
align_filter.SetUseNormalizationOff()
align_filter.SetUseScalingOff()
align_filter.SetConvergence(8e-2)

align_filter.SetNumberOfInputs(len(mesh_results))
for idx in range(len(mesh_results)):
    align_filter.SetInput(idx, mesh_results[idx])
    
align_filter.Update()

align_results = list()
for idx in range(len(mesh_results)):
    align_results.append(align_filter.GetOutput(idx))
    #set_color_point_data(align_results[idx])
    set_euclidean_distance_point_data(template_mesh, align_results[idx])


In [13]:
for idx in range(len(align_results)):
    itk.meshwrite(align_results[idx], f'Output\\correspondence\\{mesh_files[idx][13:-4]}.vtk')

In [15]:
mesh_files[2]

'Input\\meshes\\902-L-mesh.vtk'

In [16]:
target_mesh = itk.meshread(mesh_files[2], itk.F)
itk.meshwrite(target_mesh, 'Input\\meshes\\902-L-mesh.obj')

In [28]:
#view(geometries=[mesh_result,target_mesh])

## Resample From Target

A common procedure for comparing correspondences across samples is to register an atlas to each mesh sample and then deform the template to align with points on the target surface. The `MeshToMeshRegistrar` class provides an interface to use ITK's [`KdTree`](https://itk.org/Doxygen/html/classitk_1_1Statistics_1_1KdTree.html) to set each template point to its nearest neighbor on the target mesh.

In [11]:
template_resampled = \
    registrar.resample_template_from_target(mesh_result, target_mesh)

itk.meshwrite(template_resampled,POINTSET_RESAMPLED_OUTPUT_FILE)

In [ ]:
#view(geometries=[target_mesh,mesh_result,template_resampled])

In [ ]:
# Clean up file output
os.remove(MEANSQUARES_OUTPUT_FILE)
os.remove(DIFFEO_OUTPUT_FILE)
os.remove(POINTSET_OUTPUT_FILE)
os.remove(POINTSET_RESAMPLED_OUTPUT_FILE)